<a href="https://colab.research.google.com/github/dungvo1801/Sage_Publisher/blob/main/VoNgocDung_Sage_Publisher_Crawl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install google-auth
%pip install google-api-python-client

In [ ]:
%pip install selenium

In [ ]:
%pip install pandas

In [ ]:
%pip install pygsheets

In [ ]:
%pip install BeautifulSoup4

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import base64
import requests
import time
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import io
import pandas as pd
from googleapiclient.http import MediaIoBaseUpload
import pygsheets
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup

In [ ]:
driver = webdriver.Chrome()
web = []
name = []
short_name = []
issn = []
eissn = []
impact = []

for page in range(0,120):
    driver = webdriver.Chrome()
    driver.get(f"https://journals.sagepub.com/action/showPublications?pageSize=100&startPage={page}")
    soup = BeautifulSoup(driver.page_source)
    x = soup.find_all('li', class_='clearfix separator search__item')

    if len(x) == 0:
        print('Hết page')
        break

    for i in x:
        data = i.find('a')
        web.append('https://journals.sagepub.com'+str(data['href']))
        name.append(data['title'])
        short_name.append(str(data['href']).split('/')[-1])
        ind = i.find_all('div','meta__row--mobile')
        print(len(ind))
        issn.append(ind[0].text.split(': ')[-1])
        try:
            eissn.append(ind[1].text.split(': ')[-1])
        except:
            eissn.append('')
    driver.quit()



In [ ]:
for i in web:
    driver = webdriver.Chrome()
    driver.get(i)
    x = BeautifulSoup(driver.page_source)
    imp = x.find('span',class_='impact-factor__text')
    try:
        impact.append(imp.text.split(':')[1])
    except:
        impact.append('')


    driver.quit()

In [ ]:
df = pd.DataFrame({'Tên tạp chí':name,'ISSN':issn,'eISSN':eissn,'Tên viết tắt':short_name,'Impact Factor':impact})

In [ ]:
df['Tên tạp chí'] = df['Tên tạp chí'].str.replace('Go to ', '')

In [ ]:
df = df.reset_index(drop=True)
df.index += 1

In [ ]:
df.head(10)

,Tên tạp chí,ISSN,eISSN,Tên viết tắt,Impact Factor
1,AATCC Journal of Research,2472-3444,2330-5517,aata,0.7
2,About Campus,1086-4822,1536-0687,acaa,
3,Abstracts in Anthropology,0001-3455,1557-5136,aaxa,
4,Academic Forensic Pathology,1925-3621,1925-3621,afpa,
5,Accounting History,1032-3732,1749-3374,acha,1.0
6,Acta Radiologica,0284-1851,1600-0455,acrc,1.3
7,Acta Radiologica Open,2058-4601,2058-4601,arrb,1.1
8,Acta Sociologica,0001-6993,1502-3869,asja,1.7
9,Action Research,1476-7503,1741-2617,arja,1.9
10,Active Learning in Higher Education,1469-7874,1741-2625,alha,5.0


In [ ]:
df.describe()

,Tên tạp chí,ISSN,eISSN,Tên viết tắt,Impact Factor
count,1193,1193,1193,1193,1193
unique,1193,1193,1182,1193,92
top,AATCC Journal of Research,2472-3444,,aata,
freq,1,1,12,1,292


In [ ]:
%pip install openpyxl


                                              0.0/250.0 kB ? eta -:--:--
     ---------                               61.4/250.0 kB 1.1 MB/s eta 0:00:01
     -------------------------------------  245.8/250.0 kB 2.5 MB/s eta 0:00:01
     -------------------------------------  245.8/250.0 kB 2.5 MB/s eta 0:00:01
     -------------------------------------  245.8/250.0 kB 2.5 MB/s eta 0:00:01
     -------------------------------------- 250.0/250.0 kB 1.2 MB/s eta 0:00:00


In [ ]:
df.to_excel("VoNgocDung_Crawl_Data_Sage_Publisher.xlsx")